In [1]:
### IMPORTANT: RESTART KERNEL AFTER UPDATING THE MODEL TO INVOKE CHANGES! ###
from model import *
from analysis import *
import pickle

print_model = False
print_analysis = False
print_experiments = False

In [2]:
settings = [
#     X    I     N   C            W
    [ 4,   2, 1000, "A", [0.3, 0.3, 0.2, 0.2]], 
    [ 4,   4, 1000, "A", [0.3, 0.3, 0.2, 0.2]],
    [ 4,   6, 1000, "A", [0.3, 0.3, 0.2, 0.2]],
    [ 4,   8, 1000, "A", [0.3, 0.3, 0.2, 0.2]],
    [ 4,  10, 1000, "A", [0.3, 0.3, 0.2, 0.2]],
    [ 4,  12, 1000, "A", [0.3, 0.3, 0.2, 0.2]],
    [ 4,  14, 1000, "A", [0.3, 0.3, 0.2, 0.2]],
    [ 4,  16, 1000, "A", [0.3, 0.3, 0.2, 0.2]],
    [ 4,  18, 1000, "A", [0.3, 0.3, 0.2, 0.2]],
    [ 4,  20, 1000, "A", [0.3, 0.3, 0.2, 0.2]],
]

settings = [
#     X    I     N   C            W
    [ 10,   2, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]],
    [ 10,   4, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]],
    [ 10,   6, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]],
    [ 10,   8, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]],
    [ 10,  10, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]],
    [ 10,  12, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]],
    [ 10,  14, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]],
    [ 10,  16, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]],
    [ 10,  18, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]],
    [ 10,  20, 1000, "A", [0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]]
]

In [3]:
R = 1000

In [5]:
for setting in settings:
    grid_results = []
    X, I, N, C, poll_results = setting
    print("X = {}, N = {}, I = {}, W = {}, {} iterations".format(X, N, I, poll_results, R))
    
    for i in range(R):             
        agendas = generate_agendas(I, X)

        if print_model: print("Agendas: ", agendas)

        supporters = generate_relative_profile(N, agendas, poll_results)
        supporter_IDs = get_supporter_IDs(supporters, X)
        no_supporters = [len(x_supp) for x_supp in supporters]
        profile = [item for sublist in supporters for item in sublist]
        
        # if print_model: print("Supporters: {}\nSupporter IDs:{}\nNumber of supporters:{}\nProfile: {}".format(supporters, supporter_IDs, no_supporters, profile) )
        
        vote_results = simulate_vote(agendas, profile)
        if print_model: print("Vote result: ", vote_results)

        median_voter = np.around(np.mean(profile, axis=0))
        if print_model: print("Median voter: ", median_voter)
        
        gold_agreement = np.mean(calculate_agreement(median_voter, profile, 1))
        if print_model: print("Gold agreement: ", gold_agreement)
            
        for b in range(I+1):
            breaking_points = generate_breakingpoints(X, I, b)
            
            if print_experiments: print("Experiment {}: Breaking points: {}".format(b, breaking_points))

            possible_coalitions = generate_coalitions(vote_results, agendas, breaking_points)
            if print_analysis: print("{} coalitions possible".format(len(possible_coalitions)))

            if len(possible_coalitions) == 0:
                grid_results.append([gold_agreement, 0, 0, 0])
                if print_analysis: print("Expected agreement: 0 - No coalitions possible")
                continue

            expected_outcomes = simulate_outcomes(vote_results, possible_coalitions, agendas, breaking_points)
            # if print_model: print("Expected Outcomes: {}".format(expected_outcomes))
            ratings = rate_coalitions(possible_coalitions, agendas, vote_results, expected_outcomes, breaking_points)
                        
            final_expected_outcomes = np.sum([expected_outcomes[c] * ratings[c] for c in range(len(possible_coalitions))], axis=0)
            if print_model: print("Expected Outcomes: {}".format(final_expected_outcomes))
            expected_agreements = calculate_agreement(final_expected_outcomes, profile, 1)
            if print_analysis: print("Expected agreement: ", np.mean(expected_agreements))

            # # # SAMPLING # # #

#             coalition_ID = form_coalition(ratings)
#             # print(coalition_ID)
#             coalition = possible_coalitions[coalition_ID]
#             # print("Parties formed coalition {}".format(possible_coalitions[coalition_ID]))
#             policy = form_policiy(coalition, agendas, vote_results, expected_outcomes[coalition_ID])
#             # print("Coalition will implement policy {}".format(policy))
#             gold_disagreements = np.sum(np.absolute(np.subtract(policy, median_voter)))
#             gold_distance = I - 2 * gold_disagreements

#             agreements = calculate_agreement(policy, profile, 1)
#             if print_analysis: print("Sampled agreement: ", np.mean(agreements))

            entropy = calculate_entropy(final_expected_outcomes)
            if print_analysis: print("Entropy: ", entropy)

            grid_results.append([gold_agreement, np.mean(expected_agreements), len(possible_coalitions), entropy])

            print(i, end='\r'.format(R))

    filename = "model_runs/ratio/10-{}_issues.pickle".format(I)
    with open(filename, 'wb') as handle:
        pickle.dump(grid_results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("saved.")


X = 4, N = 1000, I = 2, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
X = 4, N = 1000, I = 4, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
X = 4, N = 1000, I = 6, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
X = 4, N = 1000, I = 8, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
X = 4, N = 1000, I = 10, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
X = 4, N = 1000, I = 12, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
X = 4, N = 1000, I = 14, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
X = 4, N = 1000, I = 16, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
X = 4, N = 1000, I = 18, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
X = 4, N = 1000, I = 20, W = [0.3, 0.3, 0.2, 0.2], 1000 iterations
saved.
